In [91]:
# !pip install PSID --upgrade

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from matplotlib.colors import ListedColormap
from sklearn.manifold import Isomap
import scipy.io 
import scipy.stats as stats
import PSID

In [2]:
PATH_LOAD = 'F://Aac//Work//Project//Interception//Data//Monkey_C//'

neural_data = scipy.io.loadmat(PATH_LOAD+'tdc_caeser20201022001_syjSorter_peeler_zyh_NeuralDataFile_aligned_3.mat')
handtraj_data = scipy.io.loadmat(PATH_LOAD+'C1022_HandTrajGOTT.mat')

# neural_data.keys() = ['InstantaneousRate', 'TimeHistogram', 'SpikeTimeData']
st = neural_data['SpikeTimeData'] 
# st[0, 0][0][itrial][ineuron] -> spike times
# st[0, 0][1][0, itrial] -> trial annotation
# st[0, 0][2] -> unknown
# st[0, 0][3] -> elec label
# st[0, 0][4][itrial, 0] -> target speed 

num_trial = len(st[0, 0][4])
target_speed = st[0, 0][4]
print("number of trials: {}".format(num_trial))

num_neuron = len(st[0, 0][3])
pmd_idx = [i for i, e in enumerate(st[0, 0][3]) if 'elec1' in e]
num_pmd = len(pmd_idx)
m1_idx = [i for i, e in enumerate(st[0, 0][3]) if 'elec2' in e]
num_m1 = len(m1_idx)
print("number of neurons: {}, {}/{} (PMd/M1)".format(num_neuron, num_pmd, num_m1))

start_time = -200
end_time = 600
align_marker =  3 #GO
bin = 20 #ms
num_time = int((end_time-start_time)/bin) + 1
print("number of time: {}".format(num_time))


number of trials: 772
number of neurons: 136, 41/95 (PMd/M1)
number of time: 41


In [ ]:
align_time = np.array([st[0, 0][1][0, i][0, 0][5][0, 0][1][st[0, 0][1][0, i][0, 0][5][0, 0][0]==align_marker][0] for i in range(num_trial)])
# print(align_time)
bin_edge = [(i, j) for (i, j) in zip(np.arange(start_time-bin/2, end_time+bin/2, bin), 
                                     np.arange(start_time+bin/2, end_time+bin, bin))]

sc = np.zeros((num_trial, num_time, num_neuron))
for i_trial in range(num_trial):
    for i_neuron in range(num_neuron):
        aligned_st = st[0, 0][0][i_trial]*1000 - align_time[i_trial]  # ms
        for i_time in range(num_time):
           sc[i_trial, i_time, i_neuron] = np.sum(aligned_st[i_neuron][(aligned_st[i_neuron]>bin_edge[i_time][0])&
                                                                       (aligned_st[i_neuron]<=bin_edge[i_time][1])]!=0)

# print(sc)

plt.figure()
plt.imshow(sc)
plt.show()

AttributeError: module 'matplotlib.pyplot' has no attribute 'heatmap'

: 

In [89]:
np.sum(aligned_st[i_neuron][(aligned_st[i_neuron]>bin_edge[i_time][0])&
                                                                       (aligned_st[i_neuron]<=bin_edge[i_time][1])]!=0)

1

In [44]:
print(st[0, 0][1][0, 10][0, 0][5][0, 0][1])

[[2.08122005e-01 2.19440420e+00 4.45373106e+02 8.52190042e+02
  1.41885836e+03 1.76053169e+03 1.86491770e+03 2.08552389e+03
  3.08431764e+03 3.60328867e+03]]


In [ ]:
spike_time = []

for i_trial in range(num_trial):
    trial_ts = {}
    for i_neuron in range(num_neuron):
        trial_ts.update({i_neuron: nap.Ts(t=st[0, 0][0][i_trial][i_neuron].flatten(), time_units="s")})
    tsgroup = nap.TsGroup(trial_ts)
    spike_time.append(tsgroup)


In [51]:
st[0, 0][1][0, 99][0, 0][5][0, 0][1][st[0, 0][1][0, 99][0, 0][5][0, 0][0]==3][0]

785.3446756980702

In [52]:
print(spike_time[0][4], "\n")

go_time = np.array([st[0, 0][1][0, 99][0, 0][5][0, 0][1][st[0, 0][1][0, 99][0, 0][5][0, 0][0]==3][0] for i in range(num_trial)])

epochs = nap.IntervalSet(start=go_time-200, end=go_time+600, time_units="ms")
spike_count = spike_time[0].count(0.02, epochs, time_units="s")
print(spike_count)

Time (s)
0.000683512
0.063516848
0.145483509
0.198716849
0.232450172
0.280483514
0.33915019
...
1.862616897
1.892183542
1.945683479
1.96325016
1.96538353
1.989216805
1.998783469
shape: 30 

Time (s)       0    1    2    3    4  ...
-----------  ---  ---  ---  ---  ---  -----
0.595344676    0    0    0    0    0  ...
0.615344676    1    1    0    0    0  ...
0.635344676    0    0    1    0    1  ...
0.655344676    1    0    0    0    0  ...
0.675344676    0    1    0    0    0  ...
0.695344676    0    0    0    0    1  ...
0.715344676    1    0    0    1    0  ...
...
1.255344676    0    0    0    0    0  ...
1.275344676    0    0    0    0    0  ...
1.295344676    0    1    0    0    0  ...
1.315344676    0    0    0    0    0  ...
1.335344676    0    1    0    1    0  ...
1.355344676    0    0    0    1    0  ...
1.375344676    0    0    0    0    0  ...
dtype: int64, shape: (40, 136)


C:\Users\singa\AppData\Local\Temp\ipykernel_1432\1483166819.py:5: UserWarning: start is not sorted. Sorting it.
  epochs = nap.IntervalSet(start=go_time-200, end=go_time+600, time_units="ms")
C:\Users\singa\AppData\Local\Temp\ipykernel_1432\1483166819.py:5: UserWarning: end is not sorted. Sorting it.
  epochs = nap.IntervalSet(start=go_time-200, end=go_time+600, time_units="ms")
C:\Users\singa\AppData\Local\Temp\ipykernel_1432\1483166819.py:5: UserWarning: Some starts precede the previous end. Joining them!
  epochs = nap.IntervalSet(start=go_time-200, end=go_time+600, time_units="ms")


In [ ]:
cond = dt.DelayedGenerator(target_init=m1_data['tar_dir_TO_GO_MO_Sur'][:, [0]]/180*np.pi,
                           target_speed=m1_data['speed_tri'])

cond.time_dict = {
    'time_total': (m1_data['time_TO_GO_MO_Touch'][:, 3] - m1_data['time_TO_GO_MO_Touch'][:, 0]).astype(int),
    'go': (m1_data['time_TO_GO_MO_Touch'][:, 1] - m1_data['time_TO_GO_MO_Touch'][:, 0]).astype(int),
    'mo': (m1_data['time_TO_GO_MO_Touch'][:, 2] - m1_data['time_TO_GO_MO_Touch'][:, 0]).astype(int),
    'time_move': (m1_data['time_TO_GO_MO_Touch'][:, 3] - m1_data['time_TO_GO_MO_Touch'][:, 2]).astype(int),
    'time_points': [np.arange(i) for i in
                    (m1_data['time_TO_GO_MO_Touch'][:, 3] - m1_data['time_TO_GO_MO_Touch'][:, 0]).astype(int)]}

target_rad = np.zeros((num_trial, max(cond.time_dict['time_total']), 1))
for i_trial in range(num_trial):
    target_rad[i_trial, 0:cond.time_dict['time_total'][i_trial], 0] = (
            cond.condition_dict['target_speed'][i_trial, 0]/180*np.pi/1000
            * cond.time_dict['time_points'][i_trial]
            + cond.condition_dict['target_init'][i_trial, 0])
target_rad = np.mod(target_rad, np.pi * 2)
# check
go_diff = abs(np.array([target_rad[i, cond.time_dict['go'][i], 0] for i in range(num_trial)])
              - m1_data['tar_dir_TO_GO_MO_Sur'][:, 1]/180*np.pi)
mo_diff = abs(np.array([target_rad[i, cond.time_dict['mo'][i], 0] for i in range(num_trial)])
              - m1_data['tar_dir_TO_GO_MO_Sur'][:, 2]/180*np.pi)